In [17]:
import cv2 
from scipy.spatial import distance as dist
import playsound
import numpy as np
import dlib
import imutils
from imutils import face_utils
from pygame import mixer
"""
import time
import face_recognition
from threading import Thread
tutorial link : https://youtu.be/u7_34o_rs8o?si=dbzFq0jbhUL2SVCG
"""

'\nimport time\nimport face_recognition\nfrom threading import Thread\ntutorial link : https://youtu.be/u7_34o_rs8o?si=dbzFq0jbhUL2SVCG\n'

In [18]:
# Caculating the eye aspect ratio
def EAR(eye):
    #Eye points coordinates X,Y
    A = dist.euclidean(eye[1] ,eye[5])
    B = dist.euclidean(eye[2] ,eye[4])
    C = dist.euclidean(eye[0] ,eye[3])
    ear = (A+B)/(2*C)
    return ear

In [ ]:
threshold = 0.3
frame_check = 20
detect = dlib.get_frontal_face_detector() #more faster than the cascaded method & uses svm
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [ ]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]
cap = cv2.VideoCapture(0) #to select the first camera if multiple cameras exist
flag = 0


In [22]:
#mixer Should be initialized to work
mixer.init()
mixer.music.load("music.wav")

In [23]:
while True:
    ret,frame =  cap.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray , 0)
    
    for subject in subjects:
        shape = predict(gray,subject)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart: lEnd]
        rightEye = shape[rStart: rEnd]
        #calculating Eye Aspect Ratio
        leftEAR = EAR(leftEye)
        rightEAR = EAR(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        #these next 4 lines for drawing a green line around the eye
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        if ear <threshold :
            flag +=1
            print(flag)
            if flag >= frame_check:
                cv2.putText(frame, "************alert********" , 
                (10 , 30 ) ,cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255) , 2)
                
                cv2.putText(frame, "****************ALERT!****************", (10,325),
				cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                mixer.music.play()
        else: 
            flag = 0
    cv2.imshow("Frame" , frame)
    cv2.waitKey(1) #display every 1 mS1
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    
cv2.destroyAllWindows()
cap.release()

1
2
3
4
1
2
3
4
1
2
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1
2
1
2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
1
2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
1
2
3
4
5
6
7
8
9
10
11
12
1
2
1
2
3
4
5
6
1
2
1
2
1
2
1
2
